## DDA3020 Homework 2
### Instructions:
- Follow the notebook and complete the code cells marked as TODO
- Ensure your code runs successfully until the end of the notebook

In [ ]:
from os import path as osp
import numpy as np

# load data
def load_data():

    data_dir = './data'
    train_val_data_path = osp.join(data_dir, 'train_validation_data.npy')
    train_val_label_path = osp.join(data_dir, 'train_validation_label.npy')
    test_data_path = osp.join(data_dir, 'test_data.npy')
    test_label_path = osp.join(data_dir, 'test_label.npy')

    train_val_data = np.load(train_val_data_path)
    train_val_label = np.load(train_val_label_path)
    test_data = np.load(test_data_path)
    test_label = np.load(test_label_path)
    return train_val_data, train_val_label, test_data, test_label


train_validation_data, train_validation_label, test_data, test_label = load_data()

print(f'# ========== data info ============ #')
print(f'train validation data: {train_validation_data.shape}')
print(f'train validation label: {train_validation_label.shape}')
print(f'test data: {test_data.shape}')
print(f'test label: {test_label.shape}')
print(f'# ================================= #')

# ========== data info ============ #
train validation data: (1000, 100)
train validation label: (1000,)
test data: (400, 100)
test label: (400,)
# ================================= #


## Implementing the “train validation split” function module by yourself.
(Note: You cannot change the inputs (arguments) and outputs (return values) of this function to make sure that the following code part can run correctly.
You should first read all the code snippets and understand them, and then try to complete this function.
Hint: You need to split training and validation set for the two classes evenly that means contains
800 samples (Class-0 / Class-1 = 1:1) for training set and 200 samples (Class-0 / Class-1 = 1:1) for validation set in each split.)


In [ ]:
# data split for K-fold Cross-validation

def train_validation_split(K, train_val_data, train_val_label):
                          # K (int): Number of folds.
                          # train_val_data (np.ndarray): Training and validation data.
                          # train_val_label (np.ndarray): Training and validation labels.
    # TODO: ==========================
    #Implementing the “train validation split” function module by yourself. (Note: You cannot
    # change the inputs (arguments) and outputs (return values) of this function to make sure that
    # the following code part can run correctly. You should first read all the code snippets and
    # understand them, and then try to complete this function. Hint: You need to split training
    # and validation set for the two classes evenly that means contains 800 samples (Class-0 / Class1 = 1:1) for training set and 200 samples (Class-0 / Class-1 = 1:1) for validation set in each
    # split.)
    # Shuffle the data while keeping the class balance
    from sklearn.utils import shuffle

    data_class0 = train_val_data[train_val_label == 0]
    data_class1 = train_val_data[train_val_label == 1]

    label_class0 = train_val_label[train_val_label == 0]
    label_class1 = train_val_label[train_val_label == 1]

    data_class0, label_class0 = shuffle(data_class0, label_class0, random_state=42)
    data_class1, label_class1 = shuffle(data_class1, label_class1, random_state=42)

    # Number of samples per class
    n_class0 = len(data_class0)
    n_class1 = len(data_class1)

    # Calculate fold sizes
    fold_size_class0 = n_class0 // K
    fold_size_class1 = n_class1 // K

    train_datas = []
    train_labels = []
    val_datas = []
    val_labels = []

    for i in range(K):
        # Define the start and end indices for validation set
        start_class0 = i * fold_size_class0
        end_class0 = start_class0 + fold_size_class0
        start_class1 = i * fold_size_class1
        end_class1 = start_class1 + fold_size_class1

        # Validation data for this fold
        val_data_fold = np.concatenate((data_class0[start_class0:end_class0],
                                        data_class1[start_class1:end_class1]), axis=0)
        val_label_fold = np.concatenate((label_class0[start_class0:end_class0],
                                         label_class1[start_class1:end_class1]), axis=0)

        # Training data is all data not in validation fold
        train_data_fold = np.concatenate((data_class0[:start_class0], data_class0[end_class0:],
                                          data_class1[:start_class1], data_class1[end_class1:]), axis=0)
        train_label_fold = np.concatenate((label_class0[:start_class0], label_class0[end_class0:],
                                           label_class1[:start_class1], label_class1[end_class1:]), axis=0)

        # Shuffle training and validation data
        train_data_fold, train_label_fold = shuffle(train_data_fold, train_label_fold, random_state=42)
        val_data_fold, val_label_fold = shuffle(val_data_fold, val_label_fold, random_state=42)

        train_datas.append(train_data_fold)
        train_labels.append(train_label_fold)
        val_datas.append(val_data_fold)
        val_labels.append(val_label_fold)

    return train_datas, train_labels, val_datas, val_labels
          # train_datas (list of np.ndarray): List of training data for each fold.
          #     train_labels (list of np.ndarray): List of training labels for each fold.
          #     val_datas (list of np.ndarray): List of validation data for each fold.
          #     val_labels (list of np.ndarray): List of validation labels for each fold.

In [ ]:
train_datas, train_labels, validation_datas, validation_labels = train_validation_split(5, train_validation_data, train_validation_label)

In [ ]:
len(train_labels[0])

800

In [ ]:
train_labels[0]

400.0

In [ ]:
# evaluation metrics

def eva_precision(true_label, pred_label, _class):
    # "Of all the instances predicted as positive, how many were actually positive?"
    # TODO: ==========================
    TP = np.sum((pred_label == _class) & (true_label == _class))
    FP = np.sum((pred_label == _class) & (true_label != _class))
    if TP + FP == 0:
        return 0.0
    precision = TP / (TP + FP)
    return precision

def eva_recall(true_label, pred_label, _class):
    # "Of all the actual positives, how many did the model correctly identify?"
    # TODO: ==========================
    TP = np.sum((pred_label == _class) & (true_label == _class))
    FN = np.sum((pred_label != _class) & (true_label == _class))
    if TP + FN == 0:
        return 0.0
    recall = TP / (TP + FN)
    return recall

def eva_f1(true_label, pred_label, _class):
    # F1 Score is the harmonic mean of Precision and Recall. It is useful when you need to balance Precision and Recall.
    # A high F1 score means that both Precision and Recall are high. This is particularly useful in cases where the data classes are imbalanced.
    # TODO: ==========================
    precision = eva_precision(true_label, pred_label, _class)
    recall = eva_recall(true_label, pred_label, _class)
    if precision + recall == 0:
        return 0.0
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def eva_accuracy(true_label, pred_label):
    # "Of all the instances, how many did the model predict correctly?"
    # TODO: ==========================
    correct = np.sum(true_label == pred_label)
    total = len(true_label)
    accuracy = correct / total
    return accuracy

def eva_auroc(true_label, pred_label):
    #Area Under the ROC Curve.
    # AUROC measures the ability of a model to discriminate between the positive and negative classes.
    # It evaluates the model’s performance across all classification thresholds.
    # AUROC = 0.5: This means the model is no better than random guessing.
    # AUROC = 1: This means the model perfectly distinguishes between the positive and negative classes.
    # Unlike accuracy, AUROC takes into account the performance at all classification thresholds (from 0 to 1), rather than just a fixed threshold (usually 0.5).
    # This is important in imbalanced datasets where accuracy may be misleading.
    # For example, if 95% of your dataset is negative, predicting every sample as negative would result in a high accuracy (95%), but the model would fail to detect any positive class instances.

    # TODO: ==========================
    true_label = np.array(true_label)
    pred_score = np.array(pred_label)

    desc_score_indices = np.argsort(-pred_score)
    sorted_scores = pred_score[desc_score_indices]
    sorted_labels = true_label[desc_score_indices]

    # Calculate True Positive Rate (TPR) and False Positive Rate (FPR)
    TPR = []
    FPR = []
    P = np.sum(true_label == 1)
    N = np.sum(true_label == 0)
    TP = 0
    FP = 0
    prev_score = -np.inf

    for score, label in zip(sorted_scores, sorted_labels):
        if score != prev_score:
            TPR.append(TP / P if P != 0 else 0)
            FPR.append(FP / N if N != 0 else 0)
            prev_score = score
        if label == 1:
            TP += 1
        else:
            FP += 1
    TPR.append(TP / P if P != 0 else 0)
    FPR.append(FP / N if N != 0 else 0)

    # Calculate AUROC using the trapezoidal rule
    auroc = 0.0
    for i in range(1, len(TPR)):
        auroc += (FPR[i] - FPR[i-1]) * (TPR[i] + TPR[i-1]) / 2
    return auroc

def evaluation(true_label, pred_label, _class):

    precision = eva_precision(true_label, pred_label, _class)
    recall = eva_recall(true_label, pred_label, _class)
    f1 = eva_f1(true_label, pred_label, _class)
    accuracy = eva_accuracy(true_label, pred_label)
    auroc = eva_auroc(true_label, pred_label)

    return {'precision': precision, 'recall': recall, 'f1': f1, 'accuracy': accuracy, 'auroc': auroc}



In [16]:
# model training and hyper-parameters fine-tuning
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

K = 5

# hyper-parameter for logistic regression
hyper_parameters_logistic_regression = {

    # TODO: please choose different values to tune the model
    'penalty': 'l1', # ['l1', 'l2']
    #penalty='l1' applies Lasso Regularization (for sparse models).
    #penalty='l2' applies Ridge Regularization (for smoother models).
}

# hyper-parameter for SVM
hyper_parameters_svm = {

    # TODO: please choose different values to tune the model
    'C': 1e-5, # [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]
    #Small C values impose more regularization (higher bias, lower variance).
    #Large C values reduce regularization, focusing more on fitting the training data (lower bias, higher variance).
}

# obtain cross-validation set
train_datas, train_labels, validation_datas, validation_labels = train_validation_split(K, train_validation_data, train_validation_label)


for i, (train_data, train_label, validation_data, validation_label) in enumerate(zip(train_datas, train_labels, validation_datas, validation_labels)):

    print(f'# ======================= {i + 1}-th time validation ======================= #')

    # logistic regression

    print(f'Algorithm: [logistic regression] =========================')
    print(f'hyper-parameter: {hyper_parameters_logistic_regression}')
    lr_model = LogisticRegression(solver='liblinear', **hyper_parameters_logistic_regression).fit(train_data, train_label)

    # performance evaluation on validation set for tuning hyper-parameters
    pred_label1 = lr_model.predict(validation_data)
    F1_LR_0 = eva_f1(validation_label, pred_label1, _class=0)
    print(f'F1 (Val set of Class-0): {F1_LR_0:.2f}')
    F1_LR_1 = eva_f1(validation_label, pred_label1, _class=1)
    print(f'F1 (Val set of Class-1): {F1_LR_1:.2f}')
    F1_LR_average = (F1_LR_0 + F1_LR_1) / 2
    print(f'Average F1 Linear Regression: {F1_LR_average:.2f}')

    # SVM

    print(f'Algorithm: [SVM] =========================================')
    print(f'hyper-parameter: {hyper_parameters_svm}')
    svm_model = SVC(kernel='linear', **hyper_parameters_svm).fit(train_data, train_label)

    # performance evaluation on validation set for tuning hyper-parameters
    pred_label2 = svm_model.predict(validation_data)
    F1_SVM_0 = eva_f1(validation_label, pred_label2, _class=0)
    print(f'F1 (Val set of Class-0): {F1_SVM_0:.2f}')
    F1_SVM_1 = eva_f1(validation_label, pred_label2, _class=1)
    print(f'F1 (Val set of Class-1): {F1_SVM_1:.2f}')
    F1_SVM_average = (F1_SVM_0 + F1_SVM_1) / 2
    print(f'Average F1 SVM: {F1_SVM_average:.2f}')


# ======================= 1-th time validation ======================= #
Algorithm: [logistic regression] =========================
hyper-parameter: {'penalty': 'l1'}
F1 (Val set of Class-0): 0.94
F1 (Val set of Class-1): 0.94
Average F1 Linear Regression: 0.94
Algorithm: [SVM] =========================================
hyper-parameter: {'C': 1e-05}
F1 (Val set of Class-0): 0.93
F1 (Val set of Class-1): 0.93
Average F1 SVM: 0.93
# ======================= 2-th time validation ======================= #
Algorithm: [logistic regression] =========================
hyper-parameter: {'penalty': 'l1'}
F1 (Val set of Class-0): 0.92
F1 (Val set of Class-1): 0.91
Average F1 Linear Regression: 0.91
Algorithm: [SVM] =========================================
hyper-parameter: {'C': 1e-05}
F1 (Val set of Class-0): 0.95
F1 (Val set of Class-1): 0.95
Average F1 SVM: 0.95
# ======================= 3-th time validation ======================= #
Algorithm: [logistic regression] =========================
hype

In [17]:
# performance evaluation on test set

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

K = 5

# hyper-parameter penlty for logistic regression. Hint: len(penalty) = 5
penalty = [

    # TODO: the optimal parameter selection for each split
    'l1', 'l1', 'l1', 'l1', 'l1'
]


# hyper-parameter C for SVM. Hint: len(C) = 5
C = [
    # TODO: the optimal parameter selection for each split
    1e-5, 1e-5, 1e-3, 1e-5, 1e-5
]


# obtain training data
train_datas, train_labels, _, _ = train_validation_split(K, train_validation_data, train_validation_label)


for i, (train_data, train_label) in enumerate(zip(train_datas, train_labels)):

    print(f'# ======================= {i + 1}-th time validation ======================= #')

    # logistic regression

    print(f'Algorithm: [logistic regression] =========================')
    print(f'hyper-parameter: {penalty[i]}')
    lr_model = LogisticRegression(solver='liblinear', penalty=penalty[i]).fit(train_data, train_label)


    # performance evaluation on test set
    pred_label = lr_model.predict(test_data)
    results_0 = evaluation(test_label, pred_label, _class=0)
    results_1 = evaluation(test_label, pred_label, _class=1)
    print(f'Result Class 0 (Test set): {results_0}')
    print(f'Result Class 1 (Test set): {results_1}')

    # SVM

    print(f'Algorithm: [SVM] =========================================')
    print(f'hyper-parameter: {C[i]}')
    svm_model = SVC(kernel='linear', C=C[i]).fit(train_data, train_label)

    # performance evaluation on test set
    pred_label = svm_model.predict(test_data)
    results_0 = evaluation(test_label, pred_label, _class=0)
    results_1 = evaluation(test_label, pred_label, _class=1)
    print(f'Result Class 0 (Test set): {results_0}')
    print(f'Result Class 1 (Test set): {results_1}')

# ======================= 1-th time validation ======================= #
Algorithm: [logistic regression] =========================
hyper-parameter: l1
Result Class 0 (Test set): {'precision': 0.9292929292929293, 'recall': 0.92, 'f1': 0.9246231155778895, 'accuracy': 0.925, 'auroc': 0.9250000000000002}
Result Class 1 (Test set): {'precision': 0.9207920792079208, 'recall': 0.93, 'f1': 0.9253731343283582, 'accuracy': 0.925, 'auroc': 0.9250000000000002}
Algorithm: [SVM] =========================================
hyper-parameter: 1e-05
Result Class 0 (Test set): {'precision': 0.9441624365482234, 'recall': 0.93, 'f1': 0.9370277078085644, 'accuracy': 0.9375, 'auroc': 0.9374999999999998}
Result Class 1 (Test set): {'precision': 0.9310344827586207, 'recall': 0.945, 'f1': 0.9379652605459057, 'accuracy': 0.9375, 'auroc': 0.9374999999999998}
# ======================= 2-th time validation ======================= #
Algorithm: [logistic regression] =========================
hyper-parameter: l1
Result 